# Simple loss

A simple loss provides an easy way to minimize an arbitrary function where more than simply the function is known, such as when the gradient or hessian is known. It does not require a model or data, but can be used with them.

In [ ]:
import numpy as np
import zfit

zfit.run.experimental_disable_param_update(True)

Let's start with a simple example of a function to be minimized. The function is a simple quadratic function with a minimum at (1, 2, 3).

In [ ]:
def optimizefn(x):
    return (x[0] - 1) ** 2 + (x[1] - 2) ** 2 + (x[2] - 3) ** 2


def optimizefn2(x):
    return (x[0] - 1) ** 2 + (x[1] - 2) ** 2 + (x[2] - 3) ** 2

In [ ]:
minimizer = zfit.minimize.Minuit()

Simply minimizing this function fails because an `errordef` attribute is needed; the order of magnitude of the uncertainty.

In [ ]:
minimizer.minimize(optimizefn, params=[2.0, 2.2, 2.4])

In [ ]:
optimizefn.errordef = 1  # 1 for a chi2, 0.5 for a likelihood typically

In [ ]:
result = minimizer.minimize(optimizefn, params=[2.0, 2.2, 2.4])
print(result)

## Extending the loss

To add more knowledge to the loss, we can extend it with a gradient and hessian using the `SimpleLoss` class.

In [ ]:
def gradientfn(x):
    print(f"gradientfn called with x={x}")
    return 2 * (x[0] - 1), 2 * (x[1] - 2), 2 * (x[2] - 3)


def hessianfn(x):
    print(f"hessianfn called with x={x}")
    return np.array([[2., 0, 0], [0, 2., 0], [0, 0, 2.]])


params = [zfit.Parameter(f"param_{i}", 2.0 + i * 0.2) for i in range(3)]
loss = zfit.loss.SimpleLoss(func=optimizefn, gradient=gradientfn, params=params, hessian=hessianfn)

In [ ]:
loss.gradient()

In [ ]:
loss.hessian()

In [ ]:
minimizer_grad = zfit.minimize.Minuit(gradient="zfit")

In [ ]:
result_grad = minimizer_grad.minimize(loss)
print(result_grad)

In [ ]:
result.hesse(method='hesse_np', name="loss hesse")
print(result)

In [ ]:
result_grad.hesse(name="iminuit hesse3")  # default uses iminuit, nothing printed

In [ ]:
result_grad.hesse(method='hesse_np', name="loss hesse")  # uses provided hessian

In [ ]:
result_grad

In [ ]:
result_grad.errors(name="minos")
result_grad.errors(name="zfit", method="zfit_errors")
result_grad

In [ ]:
result.errors(name="minos")
result.errors(name="zfit", method="zfit_errors")
result

In [ ]:
result.hesse(name="minuit")
result

In [ ]:
loss.hessian()

In [ ]:
np.linalg.inv(loss.hessian())

In [ ]:
# Use optimizefn2 which doesn't have errordef set, so we can pass it as parameter
optimizefn2.errordef = 0.5
loss = zfit.loss.SimpleLoss(func=optimizefn2, gradient=gradientfn, params=params)

In [ ]:
loss.hessian()

In [ ]:
res = minimizer_grad.minimize(loss)

In [ ]:
res.hesse()
res.hesse(method="hesse_np", name="loss hesse")
res.errors(name="minos")
res.errors(name="zfit", method="zfit_errors")
res

In [ ]:
0.71 ** 2

In [ ]:
minu = minimizer._minuit_minimizer

In [ ]:
minu.errordef = 0.5

In [ ]:
minu.hesse()

In [ ]:
minu.covariance

In [ ]:
p = list(result.params)[0]

In [ ]:
p

In [ ]:
result.loss.value(params={p: 2.0})